# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 7

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,30-068,1990.0,2,1,109,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.1,0.1,0.1,0.07,0.05,0.12,0.07,0.12,Matches
2,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,23-034,1997.0,6,3,289,1,1,0,0,0,0,0.31,0.31,0.62,0.31,0.62,1.9,1.9,0.1,0.58,0.03,0.60,0.58,0.60,Matches
3,Che Adams\Che-Adams,eng ENG,FW,Southampton,24-115,1996.0,7,7,585,2,2,0,0,1,0,0.31,0.31,0.62,0.31,0.62,2.3,2.3,1.5,0.35,0.23,0.58,0.35,0.58,Matches
4,Tosin Adarabioyo\Tosin-Adarabioyo,eng ENG,DF,Fulham,23-042,1997.0,3,3,270,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.1,0.1,0.0,0.04,0.01,0.05,0.04,0.05,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,33-307,1987.0,2,2,180,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col='Squad')
teamStats.head()

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Squad,,,,,,,,,,,,,,,,,,
Liverpool,1,7,5,1,1,17,15,2,16,17.2,9.9,7.2,1.03,W L D W W,NaN,Mohamed Salah - 7,Alisson,NaN
Leicester City,2,7,5,0,2,17,9,8,15,11.8,7.5,4.3,0.61,W L L W W,NaN,Jamie Vardy - 7,Kasper Schmeichel,NaN
Tottenham,3,7,4,2,1,18,9,9,14,13.3,8.0,5.4,0.76,D W D W W,NaN,Son Heung-min - 8,Hugo Lloris,NaN
Everton,4,7,4,1,2,15,11,4,13,11.4,8.3,3.1,0.44,W W D L L,NaN,Dominic Calvert-Lewin - 8,Jordan Pickford,NaN
Southampton,5,7,4,1,2,14,12,2,13,7.8,8.0,-0.2,-0.02,W W D W W,NaN,Danny Ings - 5,Alex McCarthy,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)
# this is used later
df_online=1

# fetch data locally
#df = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek) + '.csv', index_col=0)
#df_online=0

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,,0,0,-2,2,0.0,596,237,NaN,,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,596,237,False,0.0,595,237,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,68,0,0,NaN,0,,37605.jpg,0.0,0,0,Özil,0.6,False,None,n,1,3,0.0,596,237,0,3295,29,48838,1414,0.0,0.0,Özil,0
2,0,0,0,0.0,0.0,0,39476,NaN,,0,0,-1,1,0.0,536,206,NaN,,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,548,206,False,0.0,543,206,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-21T10:30:18.546407Z,49,0,0,NaN,0,,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,None,n,1,3,0.0,523,198,0,10204,9,18027,270,0.0,0.0,Sokratis,0
3,0,0,51,100.0,50.0,0,41270,NaN,,0,0,0,0,9.8,295,93,4.0,,0,2,1.5,0.5,0,David,1.0,5,0,9.7,246,82,False,66.6,187,79,319,,2020-10-25T21:30:13.186040Z,55,0,0,NaN,0,,41270.jpg,1.4,0,0,Luiz Moreira Marinho,0.9,False,None,a,1,3,21.0,222,64,7,30292,778,64422,6082,0.2,1.3,David Luiz,0
4,1,1,104,100.0,100.0,2,54694,NaN,,0,0,-3,3,119.5,39,30,6.0,,0,3,4.5,4.0,8,Pierre-Emerick,4.0,7,2,39.0,39,22,False,127.4,71,26,626,,2020-10-08T09:00:12.305506Z,117,0,0,1.0,0,,54694.jpg,4.0,0,0,Aubameyang,13.8,False,None,a,1,3,143.0,41,22,28,384952,30186,2563117,34551,0.3,2.4,Aubameyang,2
5,0,0,0,100.0,100.0,0,58822,NaN,,0,0,-3,3,0.0,510,198,NaN,,0,2,0.5,0.0,0,Cédric,0.0,0,0,0.0,525,198,False,0.0,520,198,0,,2020-09-23T09:00:14.881983Z,47,0,0,NaN,0,,58822.jpg,0.0,0,0,Soares,0.2,False,None,a,1,3,0.0,488,188,0,2765,187,30505,672,0.0,0.0,Cédric,0


In [6]:
# assign proper team names for each player
team_names = np.sort(teamStats.index)
# for some reason the fpl team numbers are not alphabetical with leeds and leicester...
if (team_names[8]=='Leeds United') & (team_names[9]=='Leicester City'):
    team_names[8] = 'Leicester City'
    team_names[9] = 'Leeds United'
df['team_name'] = team_names[df['team']-1]

## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [7]:
teamStats['xG per game'] = teamStats['xG'] / teamStats['MP']
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Liverpool,1,7,5,1,1,17,15,2,16,17.2,9.9,7.2,1.03,W L D W W,NaN,Mohamed Salah - 7,Alisson,NaN,2.457143,1.414286,0.243099
Leicester City,2,7,5,0,2,17,9,8,15,11.8,7.5,4.3,0.61,W L L W W,NaN,Jamie Vardy - 7,Kasper Schmeichel,NaN,1.685714,1.071429,0.342519
Tottenham,3,7,4,2,1,18,9,9,14,13.3,8.0,5.4,0.76,D W D W W,NaN,Son Heung-min - 8,Hugo Lloris,NaN,1.900000,1.142857,0.318907
Everton,4,7,4,1,2,15,11,4,13,11.4,8.3,3.1,0.44,W W D L L,NaN,Dominic Calvert-Lewin - 8,Jordan Pickford,NaN,1.628571,1.185714,0.305528
Southampton,5,7,4,1,2,14,12,2,13,7.8,8.0,-0.2,-0.02,W W D W W,NaN,Danny Ings - 5,Alex McCarthy,NaN,1.114286,1.142857,0.318907
Wolves,6,7,4,1,2,8,8,0,13,6.4,7.3,-0.9,-0.12,L W W D W,NaN,Raúl Jiménez - 4,Rui Patrício,NaN,0.914286,1.042857,0.352446
Chelsea,7,7,3,3,1,16,9,7,12,10.0,8.4,1.6,0.23,D W D D W,NaN,Kurt Zouma Timo Werner... - 3,Edouard Mendy Kepa Arrizabalaga,NaN,1.428571,1.200000,0.301194
Aston Villa,8,6,4,0,2,15,9,6,12,10.1,8.1,2.0,0.33,W W W L L,NaN,Jack Grealish Ollie Watkins - 4,Emiliano Martínez,NaN,1.683333,1.350000,0.259240
Arsenal,9,7,4,0,3,9,7,2,12,7.6,8.2,-0.6,-0.09,L W L L W,NaN,Alexandre Lacazette - 3,Bernd Leno,NaN,1.085714,1.171429,0.309924


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players' assists based on their xA.

In [8]:
# auxiliary information and variables
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
xA_proportion

0.6037433155080214

In [9]:
df_previous_week = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek-1) + '.csv', index_col=0)
if latest_gameweek == 1:
    df_previous_week['minutes'] = 0
    df_previous_week['xG'] = np.nan
    df_previous_week['xA'] = np.nan
df_previous_week.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,xGA_week6,cleansheet_week6,xA,xA_points,xA_week6,assists_week6,adjusted points per game,xPoints week 1,xG_week1,goals_week1,xGA_week1,cleansheet_week1,xA_week1,assists_week1,xPoints week 2,xG_week2,goals_week2,xGA_week2,cleansheet_week2,xA_week2,assists_week2,xPoints week 3,xG_week3,goals_week3,xGA_week3,cleansheet_week3,xA_week3,assists_week3,xPoints week 4,xG_week4,goals_week4,xGA_week4,cleansheet_week4,xA_week4,assists_week4,xPoints week 5,xG_week5,goals_week5,xGA_week5,cleansheet_week5,xA_week5,assists_week5,next1_xP,next5_xP,next10_xP,form 5,value,valuePoints metric,xG+xA,xG+xA per game
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,NaN,0,0,-2,2,0.0,587,236,NaN,NaN,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,589,237,False,0.0,588,237,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,68,0,0,NaN,0,NaN,37605.jpg,0.0,0,0,Özil,0.7,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0.0,0.0,0,39476,NaN,NaN,0,0,-1,1,0.0,518,202,NaN,NaN,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,532,202,False,0.0,528,202,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-21T10:30:18.546407Z,49,0,0,NaN,0,NaN,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,51,75.0,100.0,0,41270,NaN,NaN,0,0,0,0,9.8,281,87,4.0,NaN,0,2,1.1,1.5,1,David,1.5,5,0,9.7,227,77,False,66.6,161,64,319,Thigh injury - 75% chance of playing,2020-10-25T21:30:13.186040Z,55,0,0,NaN,0,NaN,41270.jpg,1.4,0,0,Luiz Moreira Marinho,1.1,False,...,1.2,0.0,0.0,0.0,0.0,0.0,2.592109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,0.0,0.0,1.9,0.0,0.0,0.0,1.199148,0.0,0.0,3.0,0.0,0.0,0.0,5.274923,0.0,0.0,0.2,0.0,0.0,0.0,3.586388,0.1,0.0,1.4,0.0,0.0,0.0,2.542828,2.402468,2.467988,2.412092,0.471293,1.105279,0.1,0.0200
4,1,0,73,100.0,100.0,1,54694,NaN,NaN,-1,1,-3,3,103.1,41,31,6.0,NaN,0,3,2.0,2.0,2,Pierre-Emerick,2.0,7,1,31.4,50,27,False,93.4,101,37,540,NaN,2020-10-08T09:00:12.305506Z,117,0,0,1.0,0,NaN,54694.jpg,3.3,0,0,Aubameyang,15.2,False,...,1.2,0.0,1.1,3.3,0.1,0.0,3.480333,3.818731,0.4,1.0,0.2,1.0,0.0,0.0,5.61799,0.1,0.0,1.9,0.0,0.5,1.0,2.049787,0.0,0.0,3.0,0.0,0.0,0.0,2.818731,0.0,0.0,0.2,0.0,0.0,0.0,3.746597,0.0,0.0,1.4,0.0,0.5,0.0,3.693373,3.502464,3.390480,3.666860,0.297464,1.017485,1.9,0.3135
5,0,0,0,100.0,100.0,0,58822,NaN,NaN,-1,1,-3,3,0.0,495,196,NaN,NaN,0,2,0.0,0.0,0,Cédric,0.0,0,0,0.0,512,196,False,0.0,508,196,0,NaN,2020-09-23T09:00:14.881983Z,47,0,0,NaN,0,NaN,58822.jpg,0.0,0,0,Soares,0.2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
if latest_gameweek > 1:
    filepath = '../data/fbref/team_stats_week' + str(latest_gameweek-1) + '.csv'
    teamStats_previous_week = pd.read_csv(filepath, index_col='Squad')
    display(teamStats_previous_week.head())
elif latest_gameweek == 1:
    teamStats_previous_week = pd.DataFrame(np.nan, index=team_names, columns=['xGA'])
    display(teamStats_previous_week)

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Everton,1,6,4,1,1,14,9,5,13,10.3,5.9,4.4,0.73,W W W D L,NaN,Dominic Calvert-Lewin - 7,Jordan Pickford,NaN,1.716667,0.983333,0.374062
Liverpool,2,6,4,1,1,15,14,1,13,15.3,9.5,5.9,0.98,W W L D W,NaN,Mohamed Salah - 6,Alisson,NaN,2.550000,1.583333,0.205290
Aston Villa,3,5,4,0,1,12,5,7,12,8.4,7.2,1.2,0.24,W W W W L,NaN,Jack Grealish Ollie Watkins - 3,Emiliano Martínez,NaN,1.680000,1.440000,0.236928
Leicester City,4,6,4,0,2,13,8,5,12,8.3,6.5,1.8,0.30,W W L L W,NaN,Jamie Vardy - 6,Kasper Schmeichel,NaN,1.383333,1.083333,0.338465
Tottenham,5,6,3,2,1,16,8,8,11,11.4,7.5,3.9,0.65,W D W D W,NaN,Son Heung-min - 8,Hugo Lloris,NaN,1.900000,1.250000,0.286505


In [11]:
def points_for_goal(position):
    if position < 3:
        return 6
    elif position==3:
        return 5
    elif position==4:
        return 4

def points_for_clean_sheet(position):
    if position < 3:
        return 4
    elif position==3:
        return 1
    elif position==4:
        return 0

In [12]:
# column names to be used below
xPoints_column = 'xPoints week ' + str(latest_gameweek)
xG_column = 'xG_week' + str(latest_gameweek) 
goals_column = 'goals_week' + str(latest_gameweek) 
xGA_column = 'xGA_week' + str(latest_gameweek) 
cleansheet_column = 'cleansheet_week' + str(latest_gameweek) 
xA_column = 'xA_week' + str(latest_gameweek) 
assists_column = 'assists_week' + str(latest_gameweek) 

In [13]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]  
    penalty_attempts = playerStats.loc[indicator,'PKatt'].values[0]
    position = df.loc[ix, 'element_type']
    
    # 0.24 is the probability to miss a penalty, which incurs -2 points
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] \
                                        - points_for_goal(position) * (df.loc[ix, 'goals_scored'] - xG) \
                                        - 2 * 0.24 * penalty_attempts    
    df.loc[ix, 'xG'] = xG
    df.loc[ix, 'xG_points'] = points_for_goal(position)*xG - 2 * 0.24 * penalty_attempts
    
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xG']):
                df.loc[ix, xG_column] = df.loc[ix, 'xG'] - df_previous_week.loc[ix, 'xG']
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored'] - df_previous_week.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
            else:
                df.loc[ix, xG_column] = xG
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xG)')

In [14]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[team, 'probability no goals allowed']
    position = df.loc[ix, 'element_type']
    
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] - points_for_clean_sheet(position) * \
                                ( df.loc[ix, 'clean_sheets'] - probability_cleanSheet*df.loc[ix, 'games played'] )
    df.loc[ix, 'clean_sheet_points'] = points_for_clean_sheet(position) * \
                                       df.loc[ix, 'games played'] * probability_cleanSheet
    try:
        if ~np.isnan(teamStats_previous_week.loc[team, 'xGA']):            
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA'] - teamStats_previous_week.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets'] - df_previous_week.loc[ix, 'clean_sheets']
        else:
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets']
        
        if df.loc[ix,'minutes'] - df_previous_week.loc[ix,'minutes'] >= 60:
            probability_cleansheet_thisweek = poisson.pmf(0,df.loc[ix, xGA_column])
            df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - points_for_clean_sheet(position) * \
                                            (df.loc[ix, cleansheet_column] - probability_cleansheet_thisweek)
        
    except KeyError:  
        print(str(ix) + ' is a new index. (xGA)')

In [15]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)
    df.loc[ix, 'xA'] = xA
    df.loc[ix, 'xA_points'] = 3*xA
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xA']):
                df.loc[ix, xA_column] = df.loc[ix, 'xA'] - df_previous_week.loc[ix, 'xA']
                df.loc[ix, assists_column] = df.loc[ix, 'assists'] - df_previous_week.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                    3 * (xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
            else:
                df.loc[ix, xA_column] = xA
                df.loc[ix, assists_column] = df.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                            3*(xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xA)')

In [16]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [17]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','elliott','mattylongstaff',"n'lundulu"]
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'mattylongstaff':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.contains('matthew longstaff'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == "n'lundulu":
                exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('lundulu')
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

597 is a new index. (xG)
597 is a new index. (xGA)
597 is a new index. (xA)


In [18]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

if (latest_gameweek > 1) & (df_online==1):
    column_list = []
    for i in range(1,latest_gameweek):
        column_list.append('xPoints week ' + str(i))
        column_list.append('xG_week' + str(i))
        column_list.append('goals_week' + str(i))
        column_list.append('xGA_week' + str(i))
        column_list.append('cleansheet_week' + str(i))
        column_list.append('xA_week' + str(i))
        column_list.append('assists_week' + str(i))
    df = df.join( df_previous_week[column_list])

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,xA,xA_points,xG_week7,goals_week7,xPoints week 7,xA_week7,assists_week7,adjusted points per game,xPoints week 1,xG_week1,goals_week1,xGA_week1,cleansheet_week1,xA_week1,assists_week1,xPoints week 2,xG_week2,goals_week2,xGA_week2,cleansheet_week2,xA_week2,assists_week2,xPoints week 3,xG_week3,goals_week3,xGA_week3,cleansheet_week3,xA_week3,assists_week3,xPoints week 4,xG_week4,goals_week4,xGA_week4,cleansheet_week4,xA_week4,assists_week4,xPoints week 5,xG_week5,goals_week5,xGA_week5,cleansheet_week5,xA_week5,assists_week5,xPoints week 6,xG_week6,goals_week6,xGA_week6,cleansheet_week6,xA_week6,assists_week6
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,,0,0,-2,2,0.0,596,237,NaN,,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,596,237,False,0.0,595,237,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,68,0,0,NaN,0,,37605.jpg,0.0,0,0,Özil,0.6,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0.0,0.0,0,39476,NaN,,0,0,-1,1,0.0,536,206,NaN,,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,548,206,False,0.0,543,206,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-21T10:30:18.546407Z,49,0,0,NaN,0,,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,51,100.0,50.0,0,41270,NaN,,0,0,0,0,9.8,295,93,4.0,,0,2,1.5,0.5,0,David,1.0,5,0,9.7,246,82,False,66.6,187,79,319,,2020-10-25T21:30:13.186040Z,55,0,0,NaN,0,,41270.jpg,1.4,0,0,Luiz Moreira Marinho,0.9,False,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2.759696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,0.0,0.0,1.9,0.0,0.0,0.0,1.199148,0.0,0.0,3.0,0.0,0.0,0.0,5.274923,0.0,0.0,0.2,0.0,0.0,0.0,3.586388,0.1,0.0,1.4,0.0,0.0,0.0,1.000000,0.0,0.0,1.2,0.0,0.0,0.0
4,1,1,104,100.0,100.0,2,54694,NaN,,0,0,-3,3,119.5,39,30,6.0,,0,3,4.5,4.0,8,Pierre-Emerick,4.0,7,2,39.0,39,22,False,127.4,71,26,626,,2020-10-08T09:00:12.305506Z,117,0,0,1.0,0,,54694.jpg,4.0,0,0,Aubameyang,13.8,False,...,1.2,3.6,0.9,1.0,7.540818,0.1,0.0,3.996891,3.818731,0.4,1.0,0.2,1.0,0.0,0.0,5.61799,0.1,0.0,1.9,0.0,0.5,1.0,2.049787,0.0,0.0,3.0,0.0,0.0,0.0,2.818731,0.0,0.0,0.2,0.0,0.0,0.0,3.746597,0.0,0.0,1.4,0.0,0.5,0.0,4.101194,0.3,0.0,1.2,0.0,0.1,0.0
5,0,0,0,100.0,100.0,0,58822,NaN,,0,0,-3,3,0.0,510,198,NaN,,0,2,0.5,0.0,0,Cédric,0.0,0,0,0.0,525,198,False,0.0,520,198,0,,2020-09-23T09:00:14.881983Z,47,0,0,NaN,0,,58822.jpg,0.0,0,0,Soares,0.2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
388,Kane,7.029703,71,10.1,62.950160,8.954882
224,Vardy,5.978261,55,9.2,47.357540,7.921625
251,Mané,6.029412,41,6.8,44.143285,7.321325
254,Salah,6.987952,58,8.3,50.847536,7.276458
563,Ait Nouri,1.000000,15,15.0,7.009785,7.009785
506,Wilson,6.973684,53,7.6,48.737540,6.988779
390,Son,7.029703,71,10.1,48.019358,6.830923
37,Grealish,6.000000,54,9.0,40.799292,6.799882
33,Hourihane,2.000000,17,8.5,13.107251,6.553625


In [20]:
# save data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [21]:
total_xG = playerStats['xG'].sum()
total_goals = df['goals_scored'].sum()
print('Total goals: ' + str(total_goals))
print('Total xG: ' + str(total_xG))
print('goals per xG: ' + str(total_goals/total_xG))

Total goals: 215
Total xG: 181.0
goals per xG: 1.1878453038674033
